# Raizen challenge - Data Analysis
### by: Henrique Voni

## Description
The challenge consists of forecasting the number of people that uses NYC metro using 7 years of history collected from turnstiles and stations. Most of the registries are audits that occurred every 4 hours, along with other entries (see `DESC` field description).


## TL;DR Proposed solution
The solution is proposed in two parts: the first one contains all data visualization and exploratory data analysis that guided me in the modelling part. I'd rather focus on the 4 busiest stations since (a priori) they don't share any relevant information with each other, and therefore it was generated one model per station.

### Modelling 
It worth mentioning that i don't like to plug any DNN and expect good results without understanding what the problem requires. So, i like to start with simple/classic models and then proceed adding complexity to the solution. In this case, i used ARIMA models with a good analysis on parameter discovery and an ETS model (Exponential Smoothing). In a more depth analysis, i'd apply deep networks and/or Transformers that can capture long term temporal correlations in data.

### Handling with data
Performance was a big part of this challenge. Some memory optimizations were made in order to work with the dataset, but `pandas` is not the best choice around for dealing with this amount of data. On a real case with appropriate time, i'd explore `dask` or `polars` for a better suitable solution.

Please consider that this notebook requires ~10Gb RAM.


## Dataset Field Description

- CA = Control Area (A002) 
- UNIT = Remote Unit for a station (R051) 
- SCP = Subunit Channel Position represents an specific address for a device (02-00-00) 
- STATION = Represents the station name the device is located at 
- LINENAME = Represents all train lines that can be boarded at this station Normally lines are represented by one character. LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains. 
- DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND
- TIME = Represents the datetime value for the registry
- DESC = Represent the REGULAR scheduled audit event (Normally occurs every 4 hours) 
    1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
    2. Additionally, there may be a RECOVR AUD entry This refers to a missed audit that was recovered. 
- ENTRIES = The comulative entry register value for a device 
- EXIST = The cumulative exit register value for a device


### If you have any questions or issues running the experiment, i'd be more than happy to help at voni.henrique@gmail.com

In [1]:
import numpy as np 
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from sklearn.metrics import mean_squared_error as MSE

from scipy.signal import detrend

sns.set_theme()

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# FUNCTIONS 
def downcast_columns(df):
    for column in df:
        if df[column].dtype == 'float64':
            df[column]=pd.to_numeric(df[column], downcast='float')
        if df[column].dtype == 'int64':
            df[column]=pd.to_numeric(df[column], downcast='integer')
    return df


def generate_train_test(df=None, station_name=None, test_date_start=None, detrendify=True):
    station_df = df.iloc[df.index.get_level_values('station') == station_name]
    
    if detrendify:
        station_df['detrended'] = detrend(station_df.traffic)

    x_train = station_df.iloc[station_df.index.get_level_values('time') < test_date_start].reset_index().detrended.astype(float)
    x_test = station_df.iloc[station_df.index.get_level_values('time') > test_date_start].reset_index().detrended.astype(float)
    return x_train, x_test

def analyze_arima_params(x_train=None):
    # parameter D
    test = adfuller(x_train)
    print('ADF Statistic: %f' % test[0])
    print('p-value: %f' % test[1])

    # parameter P
    plot_pacf(x_train.diff().dropna())
    plt.show()

    # parameter Q
    plot_acf(x_train.diff().dropna())
    plt.show()


def model_pipeline(model_name='ARIMA', p=[1,5], d=0, q=[7,11], x_train=None, x_test=None):
    history = [x for x in x_train]
    predictions = []
    
    for t in range(len(x_test)):
        if model_name == 'ARIMA':
            model = ARIMA(history, order=(p, d, q))
        else:
            model = SimpleExpSmoothing(history)

        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = x_test[t]
        history.append(obs)
    
    return history, predictions

In [3]:
INPUT_PATH = '/kaggle/input/mta-dataset'

columns = ['time', 'station', 'desc', 'entries', 'exits']
column_dtypes = {
    'station' : 'string',
    'desc' : 'string', 
    'entries' : 'Int32',
    'exits' : 'Int32'
}

df = pd.concat([pd.read_csv(file, 
                             dtype=column_dtypes, 
                             usecols=columns, 
                             parse_dates=['time']) 
                 for file 
                 in glob(f'{INPUT_PATH}/*.csv')])

df.info(memory_usage='deep')
# for file in sorted(glob(f'{INPUT_PATH}/*.csv')):
#     dfs.append(pd.read_csv(file, parse_dates=['time']).convert_dtypes())

# df = pd.concat(dfs)

# df = pd.read_csv('/kaggle/input/mta-dataset/2010.csv', parse_dates=['time']).convert_dtypes()
# df.info(memory_usage = "deep")

In [4]:
df = downcast_columns(df)

In [5]:
df.desc.value_counts()

In [6]:
df = df[df.desc == "REGULAR"]
df = df.drop('desc', axis=1)
df.shape

In [7]:
df.head()

In [8]:
by_station = df.groupby(['station', 'time'], as_index=False).agg({'entries': 'sum', 'exits':'sum'}).convert_dtypes()
del df

# Feature cleaning and processing

It was noticed that `entries` and `exits` strangely appears to have negative values. Since these negative registries are quantitatively similar to the regular values, it was assumed that it is a typo and the absolute value was considered.

Also, there were registries with null values, meaning that a turnstile could be deffective or inoperant for a moment. I decided not to treat these isolated cases with interpolation because the sum of all turnstiles of a station would still be normal.

In [9]:
by_station[['entries','exits']] = by_station[['entries','exits']].apply(lambda x: abs(x))
by_station['traffic'] = by_station['entries'] + by_station['exits']
by_station['weekday'] = by_station['time'].dt.day_name()
by_station['hour'] = by_station['time'].dt.hour

In [10]:
by_station.info()

In [11]:
by_station.traffic.sort_values(ascending=False)

In [12]:
plt.figure(figsize=(10,5))
plt.ylim(0,1)
sns.histplot(by_station.traffic, bins=10, kde=True, stat='probability')

In [13]:
by_station[by_station.traffic > 1e10 * 0.2].shape[0]

In [14]:
by_station[by_station.traffic <= 1e10 * 0.2]

In [15]:
by_station[by_station.traffic > 1e10 * 0.2].shape[0] / by_station.shape[0]

In [16]:
a = by_station.set_index('time')
a

In [17]:
traffic_by_station = by_station.groupby('station').sum().traffic

In [18]:
traffic_by_station.sort_values(ascending=False).head(50)

In [19]:
traffic_by_station.nlargest(4).index.values

In [20]:
# filter top 4 stations with most traffic
top_4_stations = traffic_by_station.nlargest(4).index.values
b = a[a.station.isin(top_4_stations)]

In [21]:
c = b.groupby(['station', 'weekday'], as_index=False).sum()

## Visualizing weekdays

The plot belows shows that business days represents the majority of volume/traffic. Curiously, the 2nd and 3rd busiest stations (25ST, CHAMBERS ST) present "more constant" values, with little gain in business days.

In [22]:
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
stations = c.station.unique()
plt.figure(figsize=(10,10))
for station in stations:
    ax = c[c.station == station].set_index('weekday').reindex(days).traffic.plot(linewidth=3, fontsize=14, rot=45, ylabel='Traffic', grid=True)    
ax.legend(labels=stations, loc='best', bbox_to_anchor=(1,1))
plt.title('Traffic per weekday', fontsize=20)
plt.show()

## Visualizing Monthly data

Sampling data in a monthly basis can help us detect some aspects such as trends, seasons, and other stuff that could be difficult to check in smaller different scales.

From the plot below, we can verify different traffic rates during the year. However, a better analysis involves decomposing the series.

In [23]:
months = ['January', 'February', 'March',
          'April', 'May', 'June', 
          'July', 'August', 'September', 
          'October', 'November', 'December']


monthly = b.groupby('station').resample('M').sum()


fig, ax = plt.subplots(figsize=(10,8))

for station, new_df in monthly.groupby(level=0, as_index=False):
    x = list(range(1,13))
    new_df = new_df.reset_index()
    new_df['month_n'] = new_df.time.dt.month
    volume_m = new_df.groupby('month_n').sum()
    fig_axis = volume_m.plot(y='traffic', rot=45, ax=ax, linewidth=3, xlabel='Month', ylabel='Traffic')
    fig_axis.set_xticks(x)
    fig_axis.set_xticklabels(months)


ax.legend(stations)
plt.show()

## Decomposition analysis

A time series can be decomposed in its base value, residuals, trend and seasonality. Decomposition comes handy when treating stationarity, detrending and other preprocessing techniques. Also, it can provide a good direction on model and parameter selection (such as ARIMA, which we will be using later).


We can see from the plots below that there's a clear trend in February with a drop in July-August. It is important to assert that our model will capture this behaviour during the forecasting task. 

In [24]:
for station, new_df in monthly.groupby(level=0, as_index=False):
    print(f'Decomposition for {station} series')
    x = list(range(0,12))
    new_df = new_df.reset_index()
    new_df['month_n'] = new_df.time.dt.month
    volume_m = new_df.groupby('month_n').sum()
    
    plt.rcParams.update({'figure.figsize': (15,10)})
    decompose_df = volume_m.reset_index().set_index('month_n')
    result_mul = seasonal_decompose(decompose_df['traffic'], 
                                    model='additive', 
                                    period=1, 
                                    extrapolate_trend='freq')
    result_mul.plot()
    plt.show()    

In [25]:
hours = list(range(24))

hourly = b.groupby(['station']).resample('H').sum().reset_index()
time = hourly.time
hourly = hourly.groupby(['station', time.dt.hour]).sum()

fig, ax = plt.subplots(figsize=(15,8))
for station, new_df in hourly.groupby(level=0, as_index=False):
    print(new_df.traffic.shape)
    fig_axis = new_df.plot(y='traffic', ax=ax, linewidth=1, linestyle='--', marker='o', xlabel='Hour', ylabel='Traffic', fontsize=16)
    fig_axis.set_xticks(hours)
    fig_axis.set_xticklabels(hours)
    
ax.legend(stations)
plt.show()

There's a curious repetitive pattern along the hours. Highest values are found during business hours, which indicates people going/returning to/from work.

In [26]:
by_station.describe()

In [27]:
daily = b.groupby('station').resample('D').sum()
daily

del by_station

In [28]:
daily.describe()

# Autocorrelation plot

Autocorrelation plot shows that temporal correlation between the series and its lags are higher for lower lags/most recent previous values.

In [29]:
fig, axs = plt.subplots(2,2, figsize=(10,8))

fig.tight_layout(h_pad=4)

for ax, (station, new_df) in zip(axs.flatten(), daily.groupby(level=0, as_index=False)):
    pd.plotting.autocorrelation_plot(new_df.traffic.tolist(), ax=ax)
    ax.set_title(station)
plt.show()

In [30]:
top_4_stations

# Modelling: forecasting 2017 data

In order to generate a model that can capture temporal patterns, i used 2010-2016 for model training and 2017 traffic as evaluation.

Personally, i like to start simple. There's no need to plug a fancy DNN and expect good results if simpler models are good options. In light of this, i proposed the usage of ARIMA and Exponential Smoothing methods. 

For ARIMA, i made a basic analysis that could guide the choice of `[p, d, q]` parameters. For `d`, i applied `Adfuller`'s hypothesis test to check stationarity. `p` and `q` were analyzed with autocorrelation and partial correlation plots. Lag values superior to the threshold area should perform better. In a real case scenario with adequate computational power, hyperparameter selection techniques such as `Grid Search` / `Random Search` could be applied.

I decided to create a model for each station, since i didn't make any correlation analysis among stations and/or used exogenous variables in training.

As an evaluation metric, i'm using Root Mean Squared Error (RMSE) that sums residuals and keep error in the same scale of input data.

# Predicting 23 ST series

In [36]:
import warnings
warnings.filterwarnings("ignore")

x_train, x_test = generate_train_test(df=daily, station_name='23 ST', test_date_start='2017-01-01')
analyze_arima_params(x_train)

history, predictions = model_pipeline(model_name='ARIMA', x_train=x_train, x_test=x_test, p=1, d=1, q=0)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ARIMA: {rmse}')


history, predictions = model_pipeline(model_name='ETS', x_train=x_train, x_test=x_test)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ETS: {rmse}')

# Predicting 125 ST series

In [37]:
x_train, x_test = generate_train_test(df=daily, station_name='125 ST', test_date_start='2017-01-01')
analyze_arima_params(x_train)

history, predictions = model_pipeline(model_name='ARIMA', x_train=x_train, x_test=x_test, p=1, d=1, q=0)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ARIMA: {rmse}')


history, predictions = model_pipeline(model_name='ETS', x_train=x_train, x_test=x_test)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ETS: {rmse}')

# Predicting CHAMBERS ST series

In [38]:
x_train, x_test = generate_train_test(df=daily, station_name='CHAMBERS ST', test_date_start='2017-01-01')
analyze_arima_params(x_train)

history, predictions = model_pipeline(model_name='ARIMA', x_train=x_train, x_test=x_test, p=1, d=1, q=0)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ARIMA: {rmse}')


history, predictions = model_pipeline(model_name='ETS', x_train=x_train, x_test=x_test)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ETS: {rmse}')

# Predicting FULTON ST series

In [39]:
x_train, x_test = generate_train_test(df=daily, station_name='FULTON ST', test_date_start='2017-01-01')
analyze_arima_params(x_train)

history, predictions = model_pipeline(model_name='ARIMA', x_train=x_train, x_test=x_test, p=1, d=1, q=0)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ARIMA: {rmse}')


history, predictions = model_pipeline(model_name='ETS', x_train=x_train, x_test=x_test)

plt.plot(x_test)
plt.plot(predictions, color='red')
plt.legend(['actual', 'predicted'])
plt.show()


rmse = np.sqrt(MSE(x_test, predictions))
print(f'RMSE ETS: {rmse}')

# Model Insights

We can see clearly for every case that ARIMA models performed better on forecasting the series when compared to a simple smoothing method because it could identify patterns/trends better, despite of exponential smoothing generating lower RMSE metrics. Regarding the parameters of ARIMA, we can conclude that:

- Adfuller hypothesis test shows that a p-value higher than 0.05 indicates the non-stationarity of the series in all cases. For sake of simplicity, i started setting ARIMA `d` parameter = `1` and got a good result. A further analysis should explore differencing techniques in order to make series stationary.

- Partial Autocorrelation and Autocorrelation plots shows highest values at `0`, which means no correlation with lag values. However, good results were achieved with `p` and `q` parameters equal to 1;

# Insight summary and further work

- MTA data has temporal nuances regarding traffic such as seasonality and trend in different sampling periods. 
- Weekend days are the less busiest days in stations;
- Hourly sampling showed higher traffic in business hours, mostly caused by people going/returning to/from work.
- Data indicates less usage of metro service in July-August and December-January. One possible cause is that July-August marks a vacation period (start of Summer) and December-January drop is related to Holidays.
- ARIMA models could provide a decent forecasting using short lag periods (as indicated in autocorrelation analysis).;
- Simpler models (ETS) couldn't converge and performed poorly on data.
- Data is non-stationary and requires detrending;
- Some registries pointed null values. Interpolation techniques should be applied in deeper investigations.
- More advanced techniques (i.e. DNNs / Transformers) should capture long term correlations and temporal patterns. An interesting idea to go deeper in this analysis involves exploring time series imaging (check https://pyts.readthedocs.io/en/stable/modules/image.htmwith) techniques and CNNs.